<a href="https://colab.research.google.com/github/JaCaRego/JaCaRego/blob/main/outliers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Tratar Outliers**


O código no caderno carrega um conjunto de dados de clientes ('clientes_limpeza.csv') e realiza várias etapas de limpeza e pré-processamento. O objetivo principal é limpar os dados, tratando outliers de idade, filtrando idades fora de um intervalo razoável e corrigindo entradas inválidas em colunas de texto como 'endereço' e 'nome'. Finalmente, os dados limpos são salvos em um novo arquivo CSV ('clientes_remove_outliers.csv').

Em resumo, as etapas realizadas são:

*   **Carregamento de Dados**: Carrega o arquivo `clientes_limpeza.csv`.
*   **Filtragem Básica**: Remove clientes com idade superior a 100 anos.
*   **Tratamento de Outliers (Z-score)**: Identifica e filtra outliers na coluna 'idade' usando o método Z-score.
*   **Tratamento de Outliers (IQR)**: Identifica e filtra outliers na coluna 'idade' usando o método do Intervalo Interquartil (IQR).
*   **Normalização de Idade**: Filtra a coluna 'idade' para garantir que os valores estejam entre 1 e 100.
*   **Tratamento de Endereços Inválidos**: Marca endereços incompletos (menos de 3 linhas) como 'Endereço inválido'.
*   **Tratamento de Nomes Longos**: Marca nomes com mais de 50 caracteres como 'Nome inválido'.
*   **Salvamento dos Dados Limpos**: Salva o DataFrame processado em um novo arquivo CSV.

In [ ]:
import pandas as pd # Importa a biblioteca pandas para manipulação de dados
from scipy import stats # Importa a biblioteca scipy para cálculos estatísticos, como o Z-score

pd.set_option('display.width', None) # Configura o pandas para exibir todas as colunas de um DataFrame

df = pd.read_csv('clientes_limpeza.csv') # Carrega o arquivo CSV 'clientes_limpeza.csv' em um DataFrame do pandas chamado 'df'

df_filtro_basico = df[df['idade'] > 100] # Cria um DataFrame 'df_filtro_basico' contendo clientes com idade superior a 100 anos

print('Filtro básico \n', df_filtro_basico[['nome','idade']]) # Imprime o nome e a idade dos clientes filtrados pelo filtro básico

# Identificar outliers com Z-score
z_scores = stats.zscore(df['idade'].dropna()) # Calcula o Z-score para a coluna 'idade', ignorando valores nulos
outliers_z = df[z_scores >= 3] # Identifica outliers como registros onde o Z-score da idade é maior ou igual a 3
print("Outliers z-score \n", outliers_z) # Imprime os registros identificados como outliers pelo método Z-score

# Filtrar outliers com Z-score
df_zscore = df[(stats.zscore(df['idade']) < 3)] # Cria um novo DataFrame 'df_zscore' removendo os outliers identificados pelo Z-score

# Identificar outliers com IQR (Intervalo Interquartil)
Q1 = df['idade'].quantile(0.25) # Calcula o primeiro quartil (Q1) da coluna 'idade'
Q3 = df['idade'].quantile(0.75) # Calcula o terceiro quartil (Q3) da coluna 'idade'
IQR = Q3 - Q1 # Calcula o Intervalo Interquartil (IQR)

limite_baixo = Q1 - 1.5 * IQR # Define o limite inferior para identificar outliers usando IQR
limite_alto = Q3 + 1.5 * IQR # Define o limite superior para identificar outliers usando IQR

print("Limites IQR: ", limite_baixo, limite_alto) # Imprime os limites superior e inferior do IQR

outliers_iqr = df[(df['idade'] < limite_baixo) | (df['idade'] > limite_alto)] # Identifica outliers como registros fora dos limites IQR
print("Outliers pelo IQR:\n ", outliers_iqr) # Imprime os registros identificados como outliers pelo método IQR

# Filtrar outliers com IQR
df_iqr = df[(df['idade'] >= limite_baixo) & (df['idade'] <= limite_alto)] # Cria um novo DataFrame 'df_iqr' removendo os outliers identificados pelo IQR

limite_baixo = 1 # Define um limite inferior de idade razoável
limite_alto = 100 # Define um limite superior de idade razoável
df = df[(df['idade'] >= limite_baixo) & (df['idade'] <= limite_alto)] # Filtra o DataFrame 'df' para manter apenas idades entre 1 e 100

# Filtrar endereços inválidos
df['endereco'] = df['endereco'].apply(lambda x: 'Endereço inválido' if len(x.split('\n')) < 3 else x) # Marca endereços com menos de 3 linhas como 'Endereço inválido'
print('Qtd registros com Endereço inválido:', (df['endereco'] == 'Endereço inválido').sum()) # Imprime a quantidade de endereços inválidos

# Tratar campos de texto (nomes longos)
df['nome'] = df['nome'].apply(lambda x: 'Nome inválido' if isinstance(x, str) and len(x) > 50 else x) # Marca nomes com mais de 50 caracteres como 'Nome inválido'
print('Qtd registros com nomes grandes:', (df['nome'] == 'Nome inválido').sum()) # Imprime a quantidade de nomes marcados como inválidos

print("Dados com Outliers tratados:\n", df) # Imprime o DataFrame final após todos os tratamentos de outliers e limpeza

# Salvar dataframe
df.to_csv('clientes_remove_outliers.csv', index=False) # Salva o DataFrame processado em um novo arquivo CSV, sem o índice

*   **Z-score (Escore Z)**:
    *   **O que é**: O Z-score é uma medida estatística que indica o quão distante um ponto de dados está da média de um conjunto de dados, em termos de desvios padrão. Um Z-score positivo significa que o ponto de dados está acima da média, enquanto um Z-score negativo indica que está abaixo da média.
    *   **Utilidade**: É uma ferramenta estatística crucial para identificar **outliers** (valores atípicos). Pontos de dados com um Z-score absoluto muito alto (geralmente acima de 2, 3 ou 3.5, dependendo do contexto) são considerados outliers, pois se desviam significativamente da maioria dos dados.
    *   **Exemplo**: Se a idade média em um grupo é 30 anos com um desvio padrão de 5 anos, uma pessoa de 45 anos teria um Z-score de (45-30)/5 = 3. Isso sugere que 45 anos é uma idade incomum para esse grupo.

*   **IQR (Intervalo Interquartil)**:
    *   **O que é**: O IQR é outra medida estatística usada para descrever a dispersão de um conjunto de dados. Ele representa a amplitude da metade central dos dados. É calculado como a diferença entre o terceiro quartil (Q3, que marca 75% dos dados) e o primeiro quartil (Q1, que marca 25% dos dados). Ou seja, IQR = Q3 - Q1.
    *   **Utilidade**: Assim como o Z-score, o IQR é uma ferramenta estatística para detectar **outliers**, sendo menos sensível a eles do que o desvio padrão. A regra comum para identificar outliers é considerar qualquer ponto de dado abaixo de `Q1 - 1.5 * IQR` ou acima de `Q3 + 1.5 * IQR` como um outlier.
    *   **Exemplo**: Se Q1 da idade é 20 e Q3 é 40, o IQR é 20. O limite inferior para outliers seria 20 - 1.5 * 20 = -10, e o limite superior seria 40 + 1.5 * 20 = 70. Qualquer idade abaixo de -10 ou acima de 70 seria considerada um outlier.

*   **`dropna()` (método do Pandas)**:
    *   **O que é**: `dropna()` é um método da biblioteca Pandas, que é uma ferramenta de manipulação de dados, não uma ferramenta estatística em si. Ele é usado para remover linhas ou colunas de um DataFrame que contêm valores nulos (NaN - Not a Number).
    *   **Utilidade**: É fundamental para o **pré-processamento de dados**. Muitas análises estatísticas e modelos de machine learning não conseguem lidar com valores nulos. Remover esses valores garante que os cálculos subsequentes sejam precisos e que o algoritmo não falhe. É importante usá-lo com cautela, pois remover muitas linhas pode levar à perda de informações valiosas.
    *   **Exemplo**: Se você tem uma coluna 'idade' com alguns valores ausentes, `df['idade'].dropna()` retornaria uma Série (ou DataFrame, dependendo do contexto) apenas com os valores de idade que não são nulos. Isso é útil antes de calcular a média ou o Z-score, pois essas operações geralmente exigem dados completos.

Aqui está um resumo dos principais comandos e suas funções no seu código:

*   **`import pandas as pd` e `from scipy import stats`**: Estas linhas importam as bibliotecas necessárias. `pandas` é essencial para manipular e analisar dados em formato de tabela (DataFrames), enquanto `scipy.stats` fornece ferramentas estatísticas, como o cálculo do Z-score.

*   **`pd.set_option('display.width', None)`**: Configura o pandas para exibir todas as colunas de um DataFrame quando impresso, evitando que o output seja truncado.

*   **`df = pd.read_csv(...)`**: Carrega o arquivo `clientes_limpeza.csv` para um DataFrame do pandas, que é a estrutura principal de dados usada para as operações subsequentes.

*   **Filtros de Idade (`df[df['idade'] > 100]`, `df[(df['idade'] >= 1) & (df['idade'] <= 100)]`)**: Estas operações são usadas para filtrar registros com base na coluna 'idade'. Elas identificam e removem idades irrealistas (como acima de 100) e garantem que as idades estejam dentro de um intervalo razoável (1 a 100).

*   **Tratamento de Outliers (Z-score e IQR)**:
    *   **`stats.zscore(df['idade'].dropna())`**: Calcula o Z-score para a coluna 'idade', uma medida estatística que indica o quão longe um ponto de dados está da média. Valores altos (geralmente >= 3) são considerados outliers.
    *   **`df['idade'].quantile(...)` e cálculos de `limite_baixo`/`limite_alto`**: Utilizam o método do Intervalo Interquartil (IQR) para identificar outliers. O IQR é a diferença entre o terceiro e o primeiro quartil, e os limites são definidos para capturar dados que fogem significativamente dessa faixa central.
    *   **Filtros baseados em Z-score e IQR**: As linhas que usam `df[z_scores >= 3]` e `df[(df['idade'] < limite_baixo) | (df['idade'] > limite_alto)]` são usadas para identificar e remover os outliers detectados por esses métodos.

*   **Tratamento de Campos de Texto (`df['endereco'].apply(...)`, `df['nome'].apply(...)`)**: Essas linhas usam a função `apply` para percorrer cada valor nas colunas 'endereco' e 'nome' e aplicar uma lógica de limpeza:
    *   Para 'endereco', verifica se o endereço tem menos de 3 linhas e o marca como 'Endereço inválido'.
    *   Para 'nome', verifica se o nome tem mais de 50 caracteres e o marca como 'Nome inválido'.

*   **`print(...)`**: Usado em várias etapas para exibir a quantidade de registros filtrados ou marcados, e também para mostrar o DataFrame final após a limpeza.

*   **`df.to_csv('clientes_remove_outliers.csv', index=False)`**: Salva o DataFrame final, já limpo e processado, em um novo arquivo CSV, sem incluir o índice do DataFrame.

In [ ]:
import pandas as pd
from scipy import stats

pd.set_option('display.width', None)

df = pd.read_csv('clientes_limpeza.csv')

df_filtro_basico = df[df['idade'] > 100]

print('Filtro básico \n', df_filtro_basico[['nome','idade']])

# Identificar outliers com Z-score
z_scores = stats.zscore(df['idade'].dropna())
outliers_z = df[z_scores >= 3]
print("Outliers z-score \n", outliers_z)

# Filtrar outliers com Z-score
df_zscore = df[(stats.zscore(df['idade']) < 3)]

# Identificar outliers com IQR
Q1 = df['idade'].quantile(0.25)
Q3 = df['idade'].quantile(0.75)
IQR = Q3 - Q1

limite_baixo = Q1 - 1.5 * IQR
limite_alto = Q3 + 1.5 * IQR

print("Limites IQR: ", limite_baixo, limite_alto)

outliers_iqr = df[(df['idade'] < limite_baixo) | (df['idade'] > limite_alto)]
print("Outliers pelo IQR:\n ", outliers_iqr)

# Filtrar outliers com IQR
df_iqr = df[(df['idade'] >= limite_baixo) & (df['idade'] <= limite_alto)]

limite_baixo = 1
limite_alto = 100
df = df[(df['idade'] >= limite_baixo) & (df['idade'] <= limite_alto)]

# Filtrar endereços inválidos
df['endereco'] = df['endereco'].apply(lambda x: 'Endereço inválido' if len(x.split('\n')) < 3 else x)
print('Qtd registros com Endereço inválido:', (df['endereco'] == 'Endereço inválido').sum())

# Tratar compos de texto
df['nome'] = df['nome'].apply(lambda x: 'Nome inválido' if isinstance(x, str) and len(x) > 50 else x)
print('Qtd registros com nomes grandes:', (df['nome'] == 'Nome inválido').sum())

print("Dados com Outliers tratados:\n", df)

# Salvar dataframe
df.to_csv('clientes_remove_outliers.csv', index=False)

